## Prepare data reader and model execution context

In [1]:
from pprint import pprint

from op_analytics.coreutils.partitioned.location import DataLocation
from op_analytics.coreutils.partitioned.reader import DataReader
from op_analytics.datapipeline.etl.ingestion.reader.byblock import construct_readers_byblock
from op_analytics.datapipeline.etl.ingestion.reader.request import BlockBatchRequest
from op_analytics.datapipeline.models.compute.markers import ModelsDataSpec
from op_analytics.datapipeline.models.compute.testutils import setup_execution_context

model_name = "account_abstraction"


# Select a model.
data_spec = ModelsDataSpec(root_path_prefix="blockbatch", models=[model_name])

# Select a block batch.
blockbatch_request = BlockBatchRequest.build(
    chains=["base"],
    range_spec="25821200:+1",
    root_paths_to_read=data_spec.input_root_paths,
)

# Construct readers
readers: list[DataReader] = construct_readers_byblock(
    blockbatch_request=blockbatch_request,
    read_from=DataLocation.GCS,
)

# Show details for the batch we are processing.
pprint(readers[0])

# Ensure existence of data needed by the reader.
assert readers[0].inputs_ready

# Set up execution context and get handles to model input args.
# In subsequent cells you can use the model input args however you want.
ctx, input_datasets, auxiliary_templates = setup_execution_context(
    model_name=model_name,
    data_reader=readers[0],  # use the first reader
)


2025-02-06 17:52:12 [debug    ] connecting to GOLDSKY Clickhouse client... filename=client.py lineno=56 process=97959
2025-02-06 17:52:12 [info     ] loaded vault from .env file    filename=vault.py lineno=32 process=97959
2025-02-06 17:52:12 [debug    ] loaded vault: 18 items         filename=vault.py lineno=79 process=97959
2025-02-06 17:52:12 [debug    ] initialized GOLDSKY Clickhouse client. filename=client.py lineno=61 process=97959
2025-02-06 17:52:13 [debug    ] connecting to OPLABS Clickhouse client... filename=client.py lineno=56 process=97959
2025-02-06 17:52:13 [debug    ] initialized OPLABS Clickhouse client. filename=client.py lineno=61 process=97959
2025-02-06 17:52:13 [info     ] prepared 2 input batches.      filename=byblock.py lineno=88 process=97959
DataReader(partitions=Partition(cols=[PartitionColumn(name='chain',
                                                      value='base'),
                                      PartitionColumn(name='dt',
                   

In [2]:
from op_analytics.datapipeline.models.code.account_abstraction.decoders import (
    register_4337_decoders,
)

register_4337_decoders(ctx)

from op_analytics.datapipeline.models.code.account_abstraction.abis import (
    INNER_HANDLE_OP_FUNCTION_METHOD_ID_v0_6_0,
    INNER_HANDLE_OP_FUNCTION_METHOD_ID_v0_7_0,
)


# Decoded UserOperationEvent logs.
user_ops = auxiliary_templates["account_abstraction/UserOperationEvent"].create_table(
    duckdb_context=ctx,
    template_parameters={
        "raw_logs": input_datasets["blockbatch/account_abstraction_prefilter/entrypoint_logs_v1"].as_subquery(),
    },
)

2025-02-06 17:52:18 [info     ] constructed read_parquet() string with 1 paths filename=client.py lineno=263 process=97959
2025-02-06 17:52:18 [info     ] Rendering query                filename=querybuilder.py lineno=40 process=97959 template=account_abstraction/UserOperationEvent
2025-02-06 17:52:19 [info     ] duck db size: 12.3KB           filename=client.py lineno=36 process=97959


In [3]:
# Traces initiated on behalf of the UserOperationEvent sender
inner_handle_op_traces = auxiliary_templates["account_abstraction/innerHandleOp"].create_table(
    duckdb_context=ctx,
    template_parameters={
        "prefiltered_traces": input_datasets["blockbatch/account_abstraction_prefilter/entrypoint_traces_v1"].as_subquery(),
        "inner_handle_op_method_ids": ", ".join(
            [
                f"'{INNER_HANDLE_OP_FUNCTION_METHOD_ID_v0_6_0}'",
                f"'{INNER_HANDLE_OP_FUNCTION_METHOD_ID_v0_7_0}'",
            ]
        ),
    },
)

2025-02-06 17:52:32 [info     ] constructed read_parquet() string with 1 paths filename=client.py lineno=263 process=97959
2025-02-06 17:52:32 [info     ] Rendering query                filename=querybuilder.py lineno=40 process=97959 template=account_abstraction/innerHandleOp
2025-02-06 17:52:33 [info     ] duck db size: 12.3KB           filename=client.py lineno=36 process=97959


In [4]:
ctx.client.sql("SHOW TABLES")

┌─────────────────────────────────────────┐
│                  name                   │
│                 varchar                 │
├─────────────────────────────────────────┤
│ account_abstraction__UserOperationEvent │
│ account_abstraction__innerHandleOp      │
└─────────────────────────────────────────┘

In [5]:
ctx.client.sql(f"SELECT * FROM {inner_handle_op_traces} LIMIT 10")

┌────────────────────────────────────────────────────────────────────┬───────────────┬────────────┬────────────────────────────────────────────┬────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────

In [6]:
ctx.client.sql(f"SELECT * FROM {user_ops} LIMIT 10")

┌────────────┬─────────┬──────────┬─────────┬─────────────────┬──────────────┬────────────────────────────────────────────────────────────────────┬────────────────────────────────────────────────────────────────────┬───────────────────┬───────────┬────────────────────────────────────────────┬────────────────────────────────────────────────────────────────────┬────────────────────────────────────────────┬────────────────────────────────────────────┬──────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────┐
│     dt     │  chain  │ chain_id │ network │ block_timestamp │ block_number │                             block_hash                             │                          transaction_hash                          │ transaction_index │ log_index │              contract_address              │                            user_op_hash                            │                   sender             

In [13]:
from op_analytics.datapipeline.models.code.account_abstraction.decoders import (
    register_4337_decoders,
)

register_4337_decoders()

from op_analytics.datapipeline.models.code.account_abstraction.abis import (
    INNER_HANDLE_OP_FUNCTION_METHOD_ID_v0_6_0,
    INNER_HANDLE_OP_FUNCTION_METHOD_ID_v0_7_0,
)

# register_4337_event_decoders(ctx)
# register_4337_function_decoders(ctx)

# Decoded UserOperationEvent logs.
user_ops_events = auxiliary_templates["account_abstraction/user_op_events"].create_table(
    duckdb_context=ctx,
    template_parameters={
        "raw_logs": input_datasets["ingestion/logs_v1"].as_subquery(),
    },
)

# Table with UserOp transaction hashes. Used to filter the raw traces.
ctx.client.sql(f"""
CREATE OR REPLACE TABLE user_op_txhash_senders AS
SELECT DISTINCT txhash_sender FROM {user_ops_events}
ORDER BY transaction_hash
""")


# Prefiltered traces.
user_op_prefiltered_traces = auxiliary_templates[
    "account_abstraction/user_op_prefiltered_traces"
].create_table(
    duckdb_context=ctx,
    template_parameters={
        "raw_traces": input_datasets["ingestion/traces_v1"].as_subquery(),
        "user_op_txhash_senders": "user_op_txhash_senders",
        "inner_handle_op_method_ids": ", ".join(
            [
                f"'{INNER_HANDLE_OP_FUNCTION_METHOD_ID_v0_6_0}'",
                f"'{INNER_HANDLE_OP_FUNCTION_METHOD_ID_v0_7_0}'",
            ]
        ),
    },
)


# Traces initiated on behalf of the UserOperationEvent sender
user_op_traces = auxiliary_templates["account_abstraction/user_op_sender_subtraces"].create_table(
    duckdb_context=ctx,
    template_parameters={
        "prefiltered_traces": user_op_prefiltered_traces,
        "inner_handle_op_method_ids": ", ".join(
            [
                f"'{INNER_HANDLE_OP_FUNCTION_METHOD_ID_v0_6_0}'",
                f"'{INNER_HANDLE_OP_FUNCTION_METHOD_ID_v0_7_0}'",
            ]
        ),
    },
)


2025-01-24 16:37:20 [info     ] constructed read_parquet() string with 1 paths filename=client.py lineno=263 process=12552
2025-01-24 16:37:20 [info     ] Rendering query                filename=querybuilder.py lineno=40 process=12552 template=account_abstraction/user_op_events
2025-01-24 16:37:30 [info     ] duck db size: 58.5MB           filename=client.py lineno=36 process=12552
2025-01-24 16:37:30 [info     ] constructed read_parquet() string with 1 paths filename=client.py lineno=263 process=12552
2025-01-24 16:37:30 [info     ] Rendering query                filename=querybuilder.py lineno=40 process=12552 template=account_abstraction/user_op_prefiltered_traces
2025-01-24 16:37:42 [info     ] duck db size: 58.5MB           filename=client.py lineno=36 process=12552
2025-01-24 16:37:42 [info     ] Rendering query                filename=querybuilder.py lineno=40 process=12552 template=account_abstraction/user_op_sender_subtraces
2025-01-24 16:37:42 [info     ] duck db size: 77.6MB

In [14]:
ctx.client.sql("SHOW TABLES")

┌─────────────────────────────────────────────────┐
│                      name                       │
│                     varchar                     │
├─────────────────────────────────────────────────┤
│ account_abstraction__user_op_events             │
│ account_abstraction__user_op_prefiltered_traces │
│ account_abstraction__user_op_sender_subtraces   │
│ user_op_txhash_senders                          │
└─────────────────────────────────────────────────┘

In [15]:
from textwrap import dedent

# Data Quality Validation. 

check1_msg = dedent("""\
    The total # of UserOperationEvent logs in a transaction must be <= the number 
    of sender subtraces found in the transaction.""")

check2_msg = dedent("""\
    We should have exactly the same UserOperationEvent logs as we do first sender
    subtrace calls.""")


check1 = auxiliary_templates["account_abstraction/data_quality_check_01"].to_relation(
    duckdb_context=ctx,
    template_parameters={}
).pl()

check2 = auxiliary_templates["account_abstraction/data_quality_check_02"].to_relation(
    duckdb_context=ctx,
    template_parameters={}
).pl()

errors = []
if len(check1) > 0:
    errors.append(check1_msg)

if len(check2) > 0:
    errors.append(check2_msg)
    
if errors:
    raise Exception("\n\n".join(errors))
else:
    print("Data Quality Check OK")


2025-01-24 16:37:58 [info     ] Rendering query                filename=querybuilder.py lineno=40 process=12552 template=account_abstraction/data_quality_check_01
2025-01-24 16:37:58 [info     ] Rendering query                filename=querybuilder.py lineno=40 process=12552 template=account_abstraction/data_quality_check_02
Data Quality Check OK


In [16]:
print(ctx.client.sql("SELECT COUNT(*) as num_events FROM account_abstraction__user_op_events"))
print(ctx.client.sql("SELECT COUNT(*) as num_traces_first_call FROM account_abstraction__user_op_sender_subtraces WHERE trace_row_number = 1 "))

print(ctx.client.sql("SELECT COUNT(*) as num_prefiltered_traces FROM account_abstraction__user_op_prefiltered_traces"))
print(ctx.client.sql("SELECT COUNT(*) as num_traces FROM account_abstraction__user_op_sender_subtraces"))

┌────────────┐
│ num_events │
│   int64    │
├────────────┤
│      11964 │
└────────────┘

┌───────────────────────┐
│ num_traces_first_call │
│         int64         │
├───────────────────────┤
│                 11964 │
└───────────────────────┘

┌────────────────────────┐
│ num_prefiltered_traces │
│         int64          │
├────────────────────────┤
│                  40359 │
└────────────────────────┘

┌────────────┐
│ num_traces │
│   int64    │
├────────────┤
│      14450 │
└────────────┘



In [17]:
ctx.client.sql("DESCRIBE TABLE account_abstraction__user_op_events")

┌──────────────────────────┬─────────────┬─────────┬─────────┬─────────┬─────────┐
│       column_name        │ column_type │  null   │   key   │ default │  extra  │
│         varchar          │   varchar   │ varchar │ varchar │ varchar │ varchar │
├──────────────────────────┼─────────────┼─────────┼─────────┼─────────┼─────────┤
│ dt                       │ DATE        │ YES     │ NULL    │ NULL    │ NULL    │
│ chain                    │ VARCHAR     │ YES     │ NULL    │ NULL    │ NULL    │
│ network                  │ VARCHAR     │ YES     │ NULL    │ NULL    │ NULL    │
│ block_timestamp          │ UINTEGER    │ YES     │ NULL    │ NULL    │ NULL    │
│ block_number             │ BIGINT      │ YES     │ NULL    │ NULL    │ NULL    │
│ block_hash               │ VARCHAR     │ YES     │ NULL    │ NULL    │ NULL    │
│ transaction_hash         │ VARCHAR     │ YES     │ NULL    │ NULL    │ NULL    │
│ transaction_index        │ BIGINT      │ YES     │ NULL    │ NULL    │ NULL    │
│ lo

In [18]:
ctx.client.sql("DESCRIBE TABLE account_abstraction__user_op_sender_subtraces")

┌───────────────────────────────┬───────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────